<a href="https://colab.research.google.com/github/Harish1208/CE156-Introduction-To-Python/blob/main/Large_Document_Summarization_Dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
pip install datasets

In [16]:
pip install evaluate

In [17]:
# Importing required libraries
import os
import pandas as pd
from datasets import Dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
import torch
import evaluate

In [18]:
# Base paths for the datasets (adjust if needed)
train_base_path = '/content/drive/MyDrive/ELITR_dataset/elitr-minuting-corpus-en/train/'
dev_base_path = '/content/drive/MyDrive/ELITR_dataset/elitr-minuting-corpus-en/dev/'
test_base_path = '/content/drive/MyDrive/ELITR_dataset/elitr-minuting-corpus-en/test/'

# Load the Excel file and read the "train" and "dev" sheets
df_train = pd.read_excel('/content/drive/MyDrive/ELITR_dataset/elitr-minuting-corpus-en/dataset_path.xlsx', sheet_name='train')
df_dev = pd.read_excel('/content/drive/MyDrive/ELITR_dataset/elitr-minuting-corpus-en/dataset_path.xlsx', sheet_name='dev')
df_test = pd.read_excel('/content/drive/MyDrive/ELITR_dataset/elitr-minuting-corpus-en/dataset_path.xlsx', sheet_name='test_1')

In [19]:
# Prepare dataset by reading the transcript and summary files
def prepare_dataset(df, base_path):
   dataset = []

   for index, row in df.iterrows():
       folder_name = row['folder_name']
       transcript_file = row['transcripts']
       summary_file = row.get('summary_1', None)  # Only for train and dev


       transcript_path = os.path.join(base_path, folder_name, transcript_file)


       with open(transcript_path, 'r', encoding='utf-8') as transcript_f:
           transcript_text = transcript_f.read()

       if summary_file:  # If summary file exists (train/dev)
           summary_path = os.path.join(base_path, folder_name, summary_file)
           with open(summary_path, 'r', encoding='utf-8') as summary_f:
               summary_text = summary_f.read()
           dataset.append({'transcript': transcript_text, 'summary': summary_text})
       else:  # For test dataset (no summary)
           dataset.append({'transcript': transcript_text})

   return Dataset.from_list(dataset)

In [20]:
# Prepare train, dev, and test datasets
train_dataset = prepare_dataset(df_train, train_base_path)
dev_dataset = prepare_dataset(df_dev, dev_base_path)
test_dataset = prepare_dataset(df_test, test_base_path)


# Load BART model and tokenizer
model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
# Tokenization function for datasets
def tokenize_function(examples):
    # Tokenize the input (transcripts)
    inputs = tokenizer(examples['transcript'], padding="max_length", truncation=True, max_length=1024)

    # Tokenize the labels (summaries) if available
    if 'summary' in examples:
        labels = tokenizer(examples['summary'], padding="max_length", truncation=True, max_length=150)
        inputs['labels'] = labels['input_ids']

    return inputs

In [22]:
# Tokenize datasets
train_tokenized = train_dataset.map(tokenize_function, batched=True)
dev_tokenized = dev_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch tensors
train_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
dev_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [29]:
# Disable WandB logging
training_args = TrainingArguments(
   output_dir='./results',
   evaluation_strategy="epoch",
   learning_rate=5e-5,
   per_device_train_batch_size=2,  # Reduce batch size for CPU
   per_device_eval_batch_size=2,   # Reduce evaluation batch size
   gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps
   num_train_epochs=3,
   weight_decay=0.01,
   logging_dir='./logs',
   fp16=False,  # Set to False as you're running on CPU
   report_to="none"  # Disable WandB logging
)

# Continue with Trainer setup and training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=dev_tokenized,
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
0,No log,2.883150
2,No log,2.580654


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'eval_loss': 2.5806541442871094,
 'eval_runtime': 0.4177,
 'eval_samples_per_second': 23.942,
 'eval_steps_per_second': 11.971,
 'epoch': 2.857142857142857}

In [30]:
# Tokenize test dataset (no labels)
test_tokenized = test_dataset.map(
    lambda examples: tokenizer(examples['transcript'], padding="max_length", truncation=True, max_length=1024),
    batched=True
)
test_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask'])

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

In [31]:
# Generate summaries for the test set
def generate_summary(test_data):
    inputs = tokenizer(test_data['transcript'], return_tensors="pt", max_length=1024, truncation=True)
    input_ids = inputs['input_ids'].to(model.device)
    attention_mask = inputs['attention_mask'].to(model.device)

    # Generate summary (greedy decoding)
    summary_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=150, num_beams=4)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

In [32]:
!pip install rouge_score
!pip install sacrebleu

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=4e10428f9d8ea2103a994664accae30e0ccafd0ac40b97854aaa545ec0300a4f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.5 MB/s eta 0:00:00


In [37]:
# Generate summaries for each entry in the test set
for i, test_example in enumerate(test_dataset):
    summary = generate_summary(test_example)
    print(f"Test Example {i+1}:")
    print(summary)
    print("-------------------------------------------------------")

Test Example 1:
MEETING MINUTES
Meeting Name: [ORGANIZATION1] [PROJECT2] Meeting Date: 09. 09. 2020
Meets Participants: [PERSON5], [PORGANIZEATION1], [ORIENTATION2], [PORIENTATION3]
Purpose of meeting: Discussion about the project and its progress.

Summary

• The goal for today will be to get everything running and get everything money on this full workload.
• [ORNERIZATION2] will be working on the call the whole Wednesday, and Tuesday afternoon is also very ehm likely.
	• The fingerprint map is not working, but the recognition is working.

-------------------------------------------------------
Test Example 2:
MEETING MINUTES
Attendees: [PERSON8], [PORGANIZATION3], [ORGANIZEATION6], [PROJECT1]
Meeting Name:   
 
Date: 09. 09. 2020
Meets Topic:  [PROPROJECT3],   [ORPROJECT5], 

-Purpose of meeting:  To discuss the progress of the [PROProject3] and [PROOPER5] projects.
-To discuss the status of the project.
--It appears that the [ORORIZATION7] submission is not happening, because there

In [38]:
# Define the directory to save generated summaries
output_summary_dir = '/content/drive/MyDrive/ELITR_dataset/elitr-minuting-corpus-en/test/generated_summaries/'

# Create the directory if it doesn't exist
os.makedirs(output_summary_dir, exist_ok=True)

# Function to generate and save summary for a test example
def generate_and_save_summary(test_data, index):
    # Generate the summary
    inputs = tokenizer(test_data['transcript'], return_tensors="pt", max_length=1024, truncation=True)
    input_ids = inputs['input_ids'].to(model.device)
    attention_mask = inputs['attention_mask'].to(model.device)

    # Generate the summary (beam search)
    summary_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=150, num_beams=4)
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Define the filename for the summary file
    summary_filename = f"generated_summary_{index+1}.txt"
    summary_filepath = os.path.join(output_summary_dir, summary_filename)

    # Save the generated summary to a file
    with open(summary_filepath, 'w', encoding='utf-8') as summary_file:
        summary_file.write(summary_text)

    # Return the summary file path
    return summary_filepath

# List to hold the paths of generated summaries
generated_summaries = []

# Generate summaries for each transcript in the test set and save them
for i, test_example in enumerate(test_dataset):
    summary_filepath = generate_and_save_summary(test_example, i)
    generated_summaries.append(summary_filepath)

# Add the generated summaries file paths to the 'summary_1' column in df_test
df_test['summary_1'] = generated_summaries

# Save the updated df_test with the summary file paths
df_test.to_excel('/content/drive/MyDrive/ELITR_dataset/elitr-minuting-corpus-en/dataset_path_updated_test.xlsx', index=False)

print("Summaries generated and saved successfully. df_test updated with summary file paths.")


Summaries generated and saved successfully. df_test updated with summary file paths.


In [44]:
import os
from rouge_score import rouge_scorer as rouge_scorer_module
import sacrebleu

# Define the directory where generated summaries are stored
output_summary_dir = '/content/drive/MyDrive/ELITR_dataset/elitr-minuting-corpus-en/test/generated_summaries/'

# Function to read generated summaries from files
def read_generated_summaries(output_summary_dir):
    generated_summaries = []
    for filename in sorted(os.listdir(output_summary_dir)):
        if filename.endswith('.txt'):
            filepath = os.path.join(output_summary_dir, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                summary_text = file.read()
                generated_summaries.append(summary_text)
    return generated_summaries

# Read the generated summaries
generated_summaries = read_generated_summaries(output_summary_dir)

# Assuming df_test contains the reference summaries
reference_summaries = df_test['summary_1'].tolist()  # This assumes you have the reference summaries in the same DataFrame

# Function to calculate ROUGE and BLEU scores for all examples
def calculate_metrics(reference_summaries, generated_summaries):
    total_rouge_scores = {
        'rouge1': [],
        'rouge2': [],
        'rougeL': []
    }

    # Initialize ROUGE scorer
    rouge_scorer = rouge_scorer_module.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Calculate ROUGE scores
    for ref_summary, gen_summary in zip(reference_summaries, generated_summaries):
        rouge_scores = rouge_scorer.score(ref_summary, gen_summary)
        total_rouge_scores['rouge1'].append(rouge_scores['rouge1'].fmeasure)
        total_rouge_scores['rouge2'].append(rouge_scores['rouge2'].fmeasure)
        total_rouge_scores['rougeL'].append(rouge_scores['rougeL'].fmeasure)

    # Average ROUGE scores
    avg_rouge1 = sum(total_rouge_scores['rouge1']) / len(total_rouge_scores['rouge1']) if total_rouge_scores['rouge1'] else 0
    avg_rouge2 = sum(total_rouge_scores['rouge2']) / len(total_rouge_scores['rouge2']) if total_rouge_scores['rouge2'] else 0
    avg_rougeL = sum(total_rouge_scores['rougeL']) / len(total_rouge_scores['rougeL']) if total_rouge_scores['rougeL'] else 0

    # Calculate BLEU score for the entire dataset
    corpus_bleu = sacrebleu.corpus_bleu(generated_summaries, [reference_summaries])

    return {
        'avg_rouge1': avg_rouge1,
        'avg_rouge2': avg_rouge2,
        'avg_rougeL': avg_rougeL,
        'bleu_score': corpus_bleu.score
    }

# Calculate metrics using the reference and generated summaries
metrics = calculate_metrics(reference_summaries, generated_summaries)

# Print metrics
print(f"Average ROUGE-1: {metrics['avg_rouge1']:.4f}")
print(f"Average ROUGE-2: {metrics['avg_rouge2']:.4f}")
print(f"Average ROUGE-L: {metrics['avg_rougeL']:.4f}")
print(f"Corpus BLEU score: {metrics['bleu_score']:.4f}")


Average ROUGE-1: 0.0414
Average ROUGE-2: 0.0000
Average ROUGE-L: 0.0384
Corpus BLEU score: 0.0474
